In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

# XGBoost
from xgboost import XGBClassifier

# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_tic import preprocessador_titanic


## 2. Dataload & Preprocessamento com joblib

In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_Titanic_v1.2.joblib')
PP2=temp['preprocessador']

# # =====================================================
# # 📁 1. Leitura dos dados & Separação das bases
# # =====================================================

DATA_DIR = BASE / "data" / "raw"
X_train = pd.read_csv(DATA_DIR / "X_train_raw.csv").reset_index(drop=True)
X_val  = pd.read_csv(DATA_DIR / "X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_raw.csv").values.ravel()
y_val  = pd.read_csv(DATA_DIR / "y_test_raw.csv")


# =====================================================
# 📁 2. Modelo e pipeline
# =====================================================
pipe_XGB3 = joblib.load('modelo_XGB_final_refine_v12.joblib')
pipe_XGB3.named_steps # informações sobre a pipeline

{'preprocess': preprocessador_titanic(),
 'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.5524895278238188, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric='logloss', feature_types=None,
               gamma=0.30499832889131046, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.27497923676042463,
               max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=6, max_leaves=None,
               min_child_weight=3, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=249, n_jobs=-1,
               num_parallel_tree=None, random_state=42, ...)}

In [3]:
# =====================================================
# 3.Treino & Teste
# =====================================================
cv_scores = cross_val_score(pipe_XGB3 , X_train, y_train, cv=10)
pipe_XGB3.fit(X_train, y_train)
y_pred=pipe_XGB3.predict(X_val) #validação
print(f"{'='*70}")
print(f"🎯 XGB(refine) | cvscores : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_val, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_val, y_pred))
cm=confusion_matrix(y_val, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")

🎯 XGB(refine) | cvscores : 0.8491 ± 0.0494
📊 **Acurácia no Teste**: 0.8321

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       157
           1       0.81      0.77      0.79       111

    accuracy                           0.83       268
   macro avg       0.83      0.82      0.83       268
weighted avg       0.83      0.83      0.83       268

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         137         20         
Real 1         25          86         
──────────────────────────────────────────────────────────────────────


## 3. Kaggle

In [4]:
# =====================================================
# Submissão Kaggle
# =====================================================
base = pd.read_csv("/home/akel/PycharmProjects/Kaggle/Titanic/data/raw/test.csv")

id_test = base["PassengerId"]

x_test=base.drop(columns='PassengerId')

# Previsão
y_pred=pipe_XGB3.predict(x_test)

#  DataFrame de submissão
submission = pd.DataFrame({
    "PassengerId": id_test,
    "Survived": y_pred
})

# submission_path = ("/home/akel/PycharmProjects/Kaggle/Titanic/data/processed/submission_XGB_refine_v1.2.csv")
# submission.to_csv(submission_path, index=False)
# print("✅ Arquivo de submissão salvo com sucesso!")